In [1]:
from pathlib import Path
import pandas as pd

from utils import *
from dataloaders import *
from data_preparation import *
from allResNets import *
from MobileNetV3 import *
from engine import *
from efficientNet import *
from data_transforms import *
from shutil import copy

change_to_disk()
models_path = Path('best_model/')
check_dir(models_path)

In [2]:
device = get_device()
NUM_EPOCHS = 100
model_name = "EfficientNetB0"
model = EfficientNetB0
effnetb0_results, final_effnetb0_model, best_effnetb0_model = load_results_model(model_name, models_path, model, NUM_EPOCHS, 87, device)
print("Done")

Done


In [3]:
#with the resize
image_shape = (1080, 1920)
resize_factor = 0.3
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]

transform = create_transform(resize,transf_tensor=True,normalize=True)

In [4]:
def get_prediction_name(predicted_labels):
    predicted_label = torch.argmax(torch.softmax(predicted_labels, dim=1), dim=1)
    if(predicted_label == 0 ):
        return "Very Good"
    if(predicted_label == 1):
        return "Good"
    elif(predicted_label == 2):
        return "Moderate"
    elif(predicted_label == 3):
        return "Poor"
    elif(predicted_label == 4):
        return "Very Poor"

In [5]:
video_path = 'cuts2.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Create a VideoWriter object to save the annotated frames
output_path = 'labeled_video2_first.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, frame_height - 50)
fontScale = 2
color = (0,0,255)
thickness = 4

# Iterate over each frame in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    pil_frame = Image.fromarray(np.uint8(frame))

    img = transform(pil_frame)
    pred_label = final_effnetb0_model(img.unsqueeze(dim=0))
    label = get_prediction_name(pred_label)

    image = cv2.putText(frame, label, org, font, fontScale, color, thickness, cv2.LINE_AA)

    out.write(image)

cap.release()
out.release()